## Remove unused columns, rows with missing info, etc.

In [1]:
import pandas as pd

#### Read metadata

In [2]:
wells = pd.read_csv("../../datasets/metadata/well.csv.gz")
orf = pd.read_csv("../../datasets/metadata/orf.csv.gz")

#### Read profiles

In [3]:
profiles = pd.read_parquet("input/raw_profiles.parquet")
profiles

,Metadata_Source,Metadata_Plate,Metadata_Well,Cells_AreaShape_Area,Cells_AreaShape_BoundingBoxArea,Cells_AreaShape_BoundingBoxMaximum_X,Cells_AreaShape_BoundingBoxMaximum_Y,Cells_AreaShape_BoundingBoxMinimum_X,Cells_AreaShape_BoundingBoxMinimum_Y,Cells_AreaShape_Center_X,...,Nuclei_Texture_Variance_RNA_5_00_256,Nuclei_Texture_Variance_RNA_5_01_256,Nuclei_Texture_Variance_RNA_5_02_256,Nuclei_Texture_Variance_RNA_5_03_256,dataset,source,workspace,profiles,batch,plate
0,source_4,BR00117035,A01,5834.000000,12214.000000,569.679993,552.330017,460.510010,444.850006,515.229980,...,34.516998,35.355000,34.396000,35.499001,2021_04_26_Batch1,BR00117035,None,None,None,None
1,source_4,BR00117035,A02,5463.799805,11298.000000,573.630005,521.140015,469.579987,416.619995,521.219971,...,41.456001,42.639999,41.368000,42.675999,2021_04_26_Batch1,BR00117035,None,None,None,None
2,source_4,BR00117035,A03,5416.000000,10838.000000,602.969971,526.530029,501.630005,423.839996,551.570007,...,43.127998,44.056000,43.116001,44.318001,2021_04_26_Batch1,BR00117035,None,None,None,None
3,source_4,BR00117035,A04,5949.799805,12099.000000,606.369995,542.229980,498.839996,433.579987,552.299988,...,48.040001,49.353001,47.851002,49.601002,2021_04_26_Batch1,BR00117035,None,None,None,None
4,source_4,BR00117035,A05,5820.700195,11727.000000,574.479980,538.989990,469.359985,430.269989,521.330017,...,37.455002,38.362000,37.421001,38.554001,2021_04_26_Batch1,BR00117035,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106347,source_4,BR00127149,P20,3994.500000,7444.100098,585.150024,593.719971,499.470001,508.179993,541.820007,...,48.408001,49.757999,48.396000,49.648998,2021_08_30_Batch13,BR00127149,None,None,None,None
106348,source_4,BR00127149,P21,3768.500000,7743.100098,574.130005,595.590027,484.209991,510.839996,528.650024,...,43.914001,44.806000,43.602001,45.058998,2021_08_30_Batch13,BR00127149,None,None,None,None
106349,source_4,BR00127149,P22,3907.800049,7337.000000,558.799988,587.570007,474.859985,502.049988,516.130005,...,46.410999,47.698002,46.439999,47.862000,2021_08_30_Batch13,BR00127149,None,None,None,None
106350,source_4,BR00127149,P23,3965.000000,7350.500000,591.299988,591.190002,506.019989,506.510010,548.289978,...,32.987000,33.810001,33.090000,33.995998,2021_08_30_Batch13,BR00127149,None,None,None,None


#### Remove extra schema columns

See https://github.com/broadinstitute/position-effect-correction/pull/4#discussion_r1158626931

In [4]:
profiles.drop(columns=['dataset', 'source', 'workspace', 'profiles', 'batch', 'plate'], inplace=True)
profiles.columns

Index(['Metadata_Source', 'Metadata_Plate', 'Metadata_Well',
       'Cells_AreaShape_Area', 'Cells_AreaShape_BoundingBoxArea',
       'Cells_AreaShape_BoundingBoxMaximum_X',
       'Cells_AreaShape_BoundingBoxMaximum_Y',
       'Cells_AreaShape_BoundingBoxMinimum_X',
       'Cells_AreaShape_BoundingBoxMinimum_Y', 'Cells_AreaShape_Center_X',
       ...
       'Nuclei_Texture_Variance_RNA_10_02_256',
       'Nuclei_Texture_Variance_RNA_10_03_256',
       'Nuclei_Texture_Variance_RNA_3_00_256',
       'Nuclei_Texture_Variance_RNA_3_01_256',
       'Nuclei_Texture_Variance_RNA_3_02_256',
       'Nuclei_Texture_Variance_RNA_3_03_256',
       'Nuclei_Texture_Variance_RNA_5_00_256',
       'Nuclei_Texture_Variance_RNA_5_01_256',
       'Nuclei_Texture_Variance_RNA_5_02_256',
       'Nuclei_Texture_Variance_RNA_5_03_256'],
      dtype='object', length=4765)

#### Join metadata and profiles

In [5]:
metadata = orf.merge(wells, on="Metadata_JCP2022")
profiles = metadata.merge(
    profiles, on=["Metadata_Source", "Metadata_Plate", "Metadata_Well"]
)
print(f"{profiles.shape=}")
profiles.groupby("Metadata_Plate")["Metadata_Well"].count()

profiles.shape=(86699, 4777)


Metadata_Plate
BR00117035    366
BR00117036    366
BR00117037    368
BR00117038    368
BR00117039    368
             ... 
BR00126714    368
BR00126715    368
BR00126716    368
BR00126717    368
BR00126718    368
Name: Metadata_Well, Length: 237, dtype: int64

#### Remove poscons, bad constructus, and rows with missing targets

In [6]:
profiles = profiles[
    ~profiles.Metadata_Symbol.isnull()
    & (profiles.Metadata_broad_sample != "BAD CONSTRUCT")
    & (profiles.Metadata_pert_type != "poscon")
]
profiles

,Metadata_JCP2022,Metadata_broad_sample,Metadata_Name,Metadata_Vector,Metadata_Transcript,Metadata_Symbol,Metadata_NCBI_Gene_ID,Metadata_Taxon_ID,Metadata_Gene_Description,Metadata_Prot_Match,...,Nuclei_Texture_Variance_RNA_10_02_256,Nuclei_Texture_Variance_RNA_10_03_256,Nuclei_Texture_Variance_RNA_3_00_256,Nuclei_Texture_Variance_RNA_3_01_256,Nuclei_Texture_Variance_RNA_3_02_256,Nuclei_Texture_Variance_RNA_3_03_256,Nuclei_Texture_Variance_RNA_5_00_256,Nuclei_Texture_Variance_RNA_5_01_256,Nuclei_Texture_Variance_RNA_5_02_256,Nuclei_Texture_Variance_RNA_5_03_256
0,JCP2022_900002,ccsbBroad304_00001,ORF008415.1_TRC304.1,pLX_304,NM_001160173.3,NAT1,9,9606,N-acetyltransferase 1,100.0,...,82.535004,82.875999,76.996002,77.473999,76.582001,77.233002,78.186996,80.055000,77.632004,79.955002
1,JCP2022_900002,ccsbBroad304_00001,ORF008415.1_TRC304.1,pLX_304,NM_001160173.3,NAT1,9,9606,N-acetyltransferase 1,100.0,...,72.304001,70.678001,67.073997,67.804001,67.042000,67.779999,68.393997,70.334999,68.253998,70.283997
2,JCP2022_900002,ccsbBroad304_00001,ORF008415.1_TRC304.1,pLX_304,NM_001160173.3,NAT1,9,9606,N-acetyltransferase 1,100.0,...,91.155998,89.515999,84.224998,84.994003,84.225998,85.214996,85.594002,88.263000,85.778000,88.174004
3,JCP2022_900002,ccsbBroad304_00001,ORF008415.1_TRC304.1,pLX_304,NM_001160173.3,NAT1,9,9606,N-acetyltransferase 1,100.0,...,71.458000,70.707001,66.263000,67.061996,66.330002,67.092003,67.583000,69.570000,67.693001,69.511002
4,JCP2022_900002,ccsbBroad304_00001,ORF008415.1_TRC304.1,pLX_304,NM_001160173.3,NAT1,9,9606,N-acetyltransferase 1,100.0,...,62.820999,61.692001,58.573002,58.917000,58.855999,59.345001,59.471001,61.160000,59.660000,61.311001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84668,JCP2022_915131,ccsbBroad304_99994,ORFC00004.1_TRC304.1,pLX_304,LacZ.1,LacZ,LacZ,CONTROL,Hahn Lab LacZ,100.0,...,64.469002,64.225998,60.463001,60.681000,60.564999,60.943001,61.087002,62.473000,61.266998,62.806000
84669,JCP2022_915131,ccsbBroad304_99994,ORFC00004.1_TRC304.1,pLX_304,LacZ.1,LacZ,LacZ,CONTROL,Hahn Lab LacZ,100.0,...,96.862000,97.935997,90.635002,91.029999,90.461998,90.711998,91.528000,93.558998,91.305000,93.669998
84670,JCP2022_915131,ccsbBroad304_99994,ORFC00004.1_TRC304.1,pLX_304,LacZ.1,LacZ,LacZ,CONTROL,Hahn Lab LacZ,100.0,...,56.007999,55.410999,52.296001,52.576000,52.507000,52.737000,52.817001,54.188999,53.081001,54.234001
84671,JCP2022_915131,ccsbBroad304_99994,ORFC00004.1_TRC304.1,pLX_304,LacZ.1,LacZ,LacZ,CONTROL,Hahn Lab LacZ,100.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


#### Save filtered profiles with metadata

In [7]:
profiles.to_parquet("output/raw_filtered_profiles.parquet")